---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [162]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [163]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [183]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    #Read a text file line by line
    with open('university_towns.txt') as f:
        content = f.readlines()
        print(content)
    data = []
    for item in content:
        if item[-6:-2] == 'edit':
            state = item[:-7]
        elif item.find('(') == -1:
            town = item[:-1]
            data.append([state,town])
        else:
            town = item[:item.find('(')-1]
            data.append([state,town])
            
    df = pd.DataFrame(data,columns=['State','RegionName'])

    return df

get_list_of_university_towns()

['Alabama[edit]\n', 'Auburn (Auburn University)[1]\n', 'Florence (University of North Alabama)\n', 'Jacksonville (Jacksonville State University)[2]\n', 'Livingston (University of West Alabama)[2]\n', 'Montevallo (University of Montevallo)[2]\n', 'Troy (Troy University)[2]\n', 'Tuscaloosa (University of Alabama, Stillman College, Shelton State)[3][4]\n', 'Tuskegee (Tuskegee University)[5]\n', 'Alaska[edit]\n', 'Fairbanks (University of Alaska Fairbanks)[2]\n', 'Arizona[edit]\n', 'Flagstaff (Northern Arizona University)[6]\n', 'Tempe (Arizona State University)\n', 'Tucson (University of Arizona)\n', 'Arkansas[edit]\n', 'Arkadelphia (Henderson State University, Ouachita Baptist University)[2]\n', 'Conway (Central Baptist College, Hendrix College, University of Central Arkansas)[2]\n', 'Fayetteville (University of Arkansas)[7]\n', 'Jonesboro (Arkansas State University)[8]\n', 'Magnolia (Southern Arkansas University)[2]\n', 'Monticello (University of Arkansas at Monticello)[2]\n', 'Russellv

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [165]:
def get_recession_start():
    gdp_data = pd.read_excel('gdplev.xls',skiprows=7)
    gdp_data = gdp_data.ix[:,4:6]
    for i in gdp_data.index[216:]:
        if (gdp_data.ix[i,1] > gdp_data.ix[i+1,1]) & (gdp_data.ix[i+1,1] > gdp_data.ix[i+2,1]):
            return gdp_data.ix[i,0]
        
    return None
get_recession_start()

'2008q3'

In [166]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    start = get_recession_start()
    
    gdp_data = pd.read_excel('gdplev.xls',skiprows=7)
    gdp_data = gdp_data.ix[:,4:6]
    index = gdp_data[gdp_data.ix[:,0] == start].index.tolist()[0]
    for i in gdp_data.index[index:]:
        if (gdp_data.ix[i,1] < gdp_data.ix[i+1,1]) & (gdp_data.ix[i+1,1] < gdp_data.ix[i+2,1]) & (gdp_data.ix[i+2,1] < gdp_data.ix[i+3,1]):
            return gdp_data.ix[i+2,0]
    return None
get_recession_end()

'2009q4'

In [167]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''

    start = get_recession_start()
    
    gdp_data = pd.read_excel('gdplev.xls',skiprows=7)
    gdp_data = gdp_data.ix[:,4:6]
    index = gdp_data[gdp_data.ix[:,0] == start].index.tolist()[0]
    for i in gdp_data.index[index:]:
        if (gdp_data.ix[i,1] < gdp_data.ix[i+1,1]) & (gdp_data.ix[i+1,1] < gdp_data.ix[i+2,1]):
            return gdp_data.ix[i,0]
    return None
get_recession_bottom()

'2009q2'

In [168]:
def convert_housing_data_to_quarters():
    import pandas as pd
    house = pd.read_csv('City_Zhvi_AllHomes.csv')
    col = 51 
    quarter = 1
    length = len(house.columns) #length
    
    #until run out of columns, get mean of every three columns, move to the next, keep track of quarter
    while (col < length):
        house[house.columns[col][:4]+'q'+str(quarter)]=house.ix[:,col:col+3].apply(lambda x:x.mean(),axis=1)
        if quarter == 4:
            quarter = 1
        else:
            quarter = quarter +1
        col = col + 3
    ans =(house.replace(states)
            .sort(columns=['State'])
            .set_index(['State','RegionName'])
            .ix[:,249:])  
    
    return ans


In [173]:
df = convert_housing_data_to_quarters()

/Users/tedpham/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:17: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [174]:
df

2000q1         2000q2         2000q3  \
State     RegionName                                                       
Alabama   Mulga                        NaN            NaN            NaN   
          Trussville         164400.000000  165900.000000  167600.000000   
          Satsuma             88600.000000   90966.666667   90333.333333   
          Irvington           72466.666667   74100.000000   75700.000000   
          Mount Olive        106500.000000  107333.333333  106966.666667   
          Dauphin Island     177700.000000  176466.666667  177133.333333   
          Tuscaloosa         104500.000000  106033.333333  108500.000000   
          Pleasant Grove     110000.000000  110233.333333  109000.000000   
          Edgewater                    NaN            NaN            NaN   
          Robertsdale         86833.333333   87666.666667   89000.000000   
          Elmore                       NaN            NaN            NaN   
          Crane Hill                   NaN            NaN            NaN   
          Morris             124066.666667  124500.000000  124233.333333   
          Guntersville        92933.333333   93500.000000   93600.000000   
          Hope Hull          111666.666667  111800.000000  112966.666667   
          Elberta            134700.000000  147033.333333  147166.666667   
          Alabaster          122133.333333  123066.666667  123166.666667   
          Holly Pond                   NaN            NaN            NaN   
          Mount Vernon        49533.333333   52433.333333   51200.000000   
          Union Grove         87500.000000   85566.666667   85233.333333   
          Hoover             184966.666667  186366.666667  187200.000000   
          Center Point        80966.666667   81233.333333   81500.000000   
          Seminole           113733.333333  123300.000000  128366.666667   
          Chalkville          94100.000000   94433.333333   94433.333333   
          Priceville                   NaN            NaN            NaN   
          Hazel Green                  NaN            NaN            NaN   
          Summerdale          91266.666667   91433.333333   96633.333333   
          Calera             108933.333333  110366.666667  108000.000000   
          Vance               99900.000000  103100.000000  105366.666667   
          Ralph              124166.666667  126766.666667  129233.333333   
...                                    ...            ...            ...   
Wisconsin Menomonee Falls    157733.333333  154533.333333  152366.666667   
          Brookfield         198700.000000  195333.333333  192866.666667   
          Oak Creek           96066.666667   99400.000000  104733.333333   
          Barton                       NaN            NaN            NaN   
          Roxbury            188800.000000  195866.666667  199233.333333   
          Cambria             90433.333333   91133.333333   91266.666667   
          Geneva              95466.666667   88600.000000   84433.333333   
          Sturtevant          79466.666667   77633.333333   78666.666667   
          Medina             173066.666667  176233.333333  176166.666667   
          Osborn              98333.333333   96200.000000   98666.666667   
          Sugar Creek         74166.666667   69100.000000   66066.666667   
          Utica              101600.000000  104000.000000  109700.000000   
          Rock                98933.333333  100366.666667   99900.000000   
          Eaton              110766.666667  114933.333333  121466.666667   
          Town of Eagle      169166.666667  167266.666667  165333.333333   
          Wales                        NaN            NaN            NaN   
          Lisbon             199733.333333  194700.000000  192133.333333   
          Holmen                       NaN            NaN            NaN   
          Holland            151033.333333  150500.000000  153233.333333   
          Lawrence           124066.666667  128233.333333  134466.666667   
        

In [177]:
#find the priceratio = price before recession/recession bottom
df['priceratio']=df['2008q2'].div(df['2009q2'])


In [182]:
university = get_list_of_university_towns()
niversity = get_list_of_university_towns()
    
uni_t = [tuple(x) for x in university.values]

uni_house = df.ix[uni_t]
non_uni_house = df.drop(uni_t)

uni_price = uni_house['priceratio'].dropna().tolist()
non_uni_price = non_uni_house['priceratio'].dropna().tolist()

ttest_ind(non_uni_price,uni_price)[1]

0.0027240637047608778